In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

# Complete simulation loop

1. Get target position
2. Obtain low-level updates and predictions in loop when interval is less than update interval

3. Calculate SN0
4. Calculate SNR using predicted angle coordinates
5. Calculate PD -> assume that the PD remains the same though the dwells
5. Simulate the dwell time (first expected later can be modified to stochastic)
6. return load
7. Calculate new interval for measurement
8. Return step one

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from filterpy.kalman import KalmanFilter
from filterpy.common import kinematic_kf
from target import SingerModel
from simulation import simulate_on_trajectory, create_trajectory
from tracking import ResidualUpdatePolicy
from radar import MonostaticRadar

In [25]:
def cart2polar(x):
    return np.array([np.linalg.norm(x), np.arctan(x[1]/x[0])])

def measurement_matrix(angle, var_r, var_alpha):
    var_x = var_r * np.cos(-angle)**2 + var_alpha * np.sin(-angle)**2
    var_y = var_r * np.sin(-angle)**2 + var_alpha * np.cos(-angle)**2
    var_xy = (var_r * var_alpha) * np.sin(-angle)**2 * np.cos(-angle)**2
    return np.array([
        [var_x, var_xy],
        [var_xy, var_y]
    ])

In [12]:
dt = 0.1
horizon=100
target = SingerModel(1, 1, np.zeros(3))
trajectory1 = create_trajectory(target, dt, horizon)
trajectory2 = create_trajectory(target, dt, horizon)

states = np.concatenate((trajectory1, trajectory2), axis=1)

In [19]:
tracker = kinematic_kf(dim=2, order=2)
tracker.x = np.zeros((6, 1))
tracker.F[:3, :3] = target.F(dt)
tracker.F[3:, 3:] = target.F(dt)
tracker.Q[:3, :3] = target.Q(dt)
tracker.Q[3:, 3:] = target.Q(dt)

array([[1., 0.],
       [0., 1.]])

In [ ]:
power = 100e3
gain_transmit = 1
gain_receive = 1
wavelength = 0.15
rcs = 1
thermal_noise = 0.001
horizon = 1000
beamwidth = 5/180 * np.pi
pf = 1e-5
# simulation loop



positions = np.array([trajectory1[:, 0], trajectory2[:, 0]]).T

for t in range(horizon):
    pos = positions[t] # get positions
    polar = cart2polar(pos)
    angle = np.arctan(pos[1]/pos[0])
    # Get predicted SNR at the given position 
    angle_hat
    distance = np.linalg.norm(pos)
    SN0 = (power*gain_transmit*gain_receive*wavelength**2*rcs)/((4*np.pi)**3*distance**4*thermal_noise)
    SNR = SN0 * np.exp(-2 * (angle_hat - angle)**2 / (beamwidth**2))
    Pd = pf**(1/(1+SNR))
    
    if True:
        dwell_time = 1/Pd
        load = dwell_time / interval
    else:
        pass